## 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

#### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성/, 구성 요소 등을 가진 다른 영화를 추천함
이번 예제에서는 영화 장르 속성을 기반으로 콘텐츠 기반 필터링 추천 시스템을 만듦
장르 칼럼 값의 유사도를 비교, 그 중 높은 평점을 가지는 영화 추천

### 데이터 로딩 및 가공
DataFrame 로딩

In [5]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


tmdb_5000_movies.csv : 4803개의 레코드, 20개의 피처로 구성됨
영화 제목, 개요, 인기도, 평점, 투표수, 예산 키워드 등 영화에 대한 다양한 메타 정보를 가짐

이 중에서 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출 -> 새로운 DataFrame 만듦

In [6]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

주의해야 할 칼럼 : 'genres', 'keywords' 등과 같은 칼럼은 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기됨(한꺼번에 여러 개의 값을 표현하기 위한 표기 방식)
    
하지만 이 칼럼이 DataFrame으로 만들어지면 단순히 문자열로 로딩됨 -> 칼럼을 가공해야 필요한 정보 추출 가능

In [7]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출
파이썬 ast 모듈의 literal_eval() 함수 이용

In [9]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

genres 칼럼에서 키에 해당하는 값을 추출하기 위해 apply lambda 식 이용

apply(lambda x: [y['name'] for y in x])와 같이 변환하면 리스트 내 여러 개의 딕셔너리의 'name' 키에 해당하는 값을 찾아 리스트 객체로 변환

In [11]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정
genres 칼럼은 여러 개의 개별 장르가 리스트로 구성되어 있음 -> 영화 하나의 장르가 여러 개로 되어 있다면 장르별 유사도 측정을 어떻게 할 것인가?

가장 간단한 방법은 genres를 문자열로 변경한 뒤 이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것

1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


2. 생성된 피처 벡터 행렬에 사이킷런 cosine_similarity()를 이용해 코사인 유사도 계산

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


gere_sim 객체는 dataFrame의 genre_literal 칼럼을 피처 벡터화한 행렬 데이터의 행별 유사도 정보를 가지고 있음

결국 DataFrame의 행별 장르 유사도 값을 가지고 있는 것

장르 기준 콘텐츠 기반 필터링 수행을 위해 genre_sim 객체 이용 -> 객체의 기준 행별로 비교 대상이 되는 행이 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출

argsort() 함수 이용

In [15]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


=> 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 레코드, 813 레코드 순으로 유사도가 높음.


### 장르 콘텐츠 필터링을 이용한 영화 추천
장르 유사도에 따라 영화를 추천하는 함수 생성

함수명 find_sim_movie(), 인자 : movies_df DataFrame, 레코드별 장르 코사인 유사도 인덱스 가지는 genre_sim_sorted_ind, 고객이 선정한 추천 기준이 되는 영화 제목, 추천할 영화 건수
인자를 입력하면 추천 영화 정보를 가지는 DataFrame 반환

In [16]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

find_sim_movie() 함수를 이용해 영화 '대부'와 장르별로 유사한 영화 10개 추천

In [17]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


대부와 비슷한 유형이 추천되기도 하지만 평점이 낮고 낯선 영화도 있음. -> 개선이 필요함 -> 더 많은 후보군 선정 후 영화 평점에 따라 필터링해 최종 추천

평점 정보인 'vote_average'값을 이용함 (0~10점으로 구성됨, 고객 평가 평점의 평균이므로 소수 관객의 평점으로 왜곡된 데이터 존재)

왜곡된 데이터 확인을 위해 sort_values를 이용

In [18]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


평가 횟수가 매우 작은 영화의 경우 왜곡된 평점 데이터를 지님 -> 평점에 평가 횟수를 반영하는 새로운 평가 방식 필요

유명한 영화 평점 사이트에서는 평가 횟수에 대한 가중치가 부여된 평점 방식을 사용함. 이 방식 적용.

V : movies_df의 'vote_count' 값
R : movies_df의 'vote_average' 값
C : 전체 영화의 평균 평점
m : 투표 횟수에 따른 가중치를 직접 조절하는 역할 (m값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점 부여)
    
m을 전체 투표 횟수에서 상위 60%에 해당하는 횟수 기준으로 정함

In [19]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


기존 평점을 새로운 가중 평점으로 변경하는 함수 weighted_vote_average() 생성


DataFrame의 레코드를 인자로 받아 레코드의 vote_count와 vote_average 칼럼, 미리 추출된 m과 C 값을 적용해 레코드별 가중 평점 반환

함수를 movies_df의 apply() 함수의 인자로 입력해 가중 평점 계산

In [20]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 


새롭게 부여된 weighted_vote 평점이 높은 순으로 상위 10개 영화 추출

In [21]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                          ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


새롭게 정의된 평점 기준에 따라 영화 추천
장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤 weighted_vote 칼럼 값이 높은 순으로 top_n만큼 추출하는 방식

find_sim_movie() 함수 변경 후 '대부'와 유사한 영화 콘텐츠 기반 필터링 방식으로 추천함

In [22]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]


,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


전보다 훨씬 나은 영화가 추천되었지만 장르만으로 개인의 성향을 반영하기 부족함